<a href="https://colab.research.google.com/github/UiinKim/Crawling/blob/main/keyword_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 추출
file_name = next(iter(uploaded))

# 'utf-8-sig'로 파일 읽기
df = pd.read_csv(file_name, encoding='cp949')

# 데이터프레임 확인
print(df)

Saving 김의인_레이블링.csv to 김의인_레이블링 (3).csv
       Unnamed: 0   idx brand_en brand_ko         product_name  product_num  \
0               1     1      V&A    브이앤에이  안티옥시던트 래디언스 앰플 50ml   5931851297   
1               2     2      V&A    브이앤에이  안티옥시던트 래디언스 앰플 50ml   5931851297   
2               3     3      V&A    브이앤에이  안티옥시던트 래디언스 앰플 50ml   5931851297   
3               4     4      V&A    브이앤에이  안티옥시던트 래디언스 앰플 50ml   5931851297   
4               5     5      V&A    브이앤에이  안티옥시던트 래디언스 앰플 50ml   5931851297   
...           ...   ...      ...      ...                  ...          ...   
11195       11196  2125    avene      아벤느        트릭세라 로션 100ml   7800018262   
11196       11197  2126    avene      아벤느        트릭세라 로션 100ml   7800018262   
11197       11198  2127    avene      아벤느        트릭세라 로션 100ml   7800018262   
11198       11199  2128    avene      아벤느        트릭세라 로션 100ml   7800018262   
11199       11200  2129    avene      아벤느        트릭세라 로션 100ml   7800018262   

           

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def find_similar_reviews(query, documents):
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)

    # 쿼리를 TF-IDF로 변환
    query_vector = vectorizer.transform([query])

    # 코사인 유사도 계산
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()

    # 유사도가 높은 리뷰 순으로 정렬
    similar_reviews_indices = cosine_similarities.argsort()[:-1][::-1]

    # 유사도가 높은 상위 5개 리뷰 출력
    for idx in similar_reviews_indices[:5]:
        print(f"Similarity: {cosine_similarities[idx]:.4f}")
        print(f"Review: {documents[idx]}")
        print()

# 테스트
text_query = '수분이 부족해서 수분크림을 추천해주세요'
reviews = df['review'].tolist()
find_similar_reviews(text_query, reviews)

Similarity: 0.2272
Review: 잘 발리고 수분이 오래가요

Similarity: 0.2211
Review: 흡수도 즉각빨라서 너무 좋아요 샘플까지 된다님 적게 일하시고 많이 버시고 좋은 제품 많이 추천해주세요 

Similarity: 0.2189
Review: 촉촉하고 수분이 너무 많아서너무좋아요

Similarity: 0.2101
Review: 메이크프렘 수분크림을 좋아하는데 이건 처음 구매해봅니다

Similarity: 0.1918
Review: 이때까지 많은 종류의 수분크림을 써봤는데 발림성 수분감 모두 다 만족합니다 피부가 아주 건조한 편이라서 선크림만 발라도 뜰 때가 있었는데 메이크프렘 수분크림을 바르고 선크림을 바르면 안떠서 좋았어요 재구매 의사 있습니다 

